In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dash_table import DataTable

# Wind farm details
wind_farm_capacity_mw = 312  # Capacity in MW
capacity_factor = 0.30  # 30% capacity factor
annual_energy_production_mwh = wind_farm_capacity_mw * 8760 * capacity_factor  # Annual MWh production

# Example input data with off-take tenors
ppa_offers = pd.DataFrame({
    'PPA_Price': [50, 55, 60],  # Prices in $/MWh
    'CPI': [0.02, 0.025, 0.03],  # Annual escalation rate
    'Capacity': [0.4, 0.35, 0.25],  # Capacity percentage
    'Tenor': [7, 6, 3]  # Off-take tenors in years
})

spot_prices = pd.Series([45, 46, 47, 48, 49, 50, 51, 52, 53, 54], name='Spot_Price')  # Forecasted spot prices for 10 years
years = len(spot_prices)
discount_rate = 0.08

# Initialize DataFrame to store revenue calculations
columns = ['PPA_Revenue', 'Spot_Revenue', 'Total_Revenue', 'Discount_Factor', 'Discounted_Revenue']
total_revenue = pd.DataFrame(index=[f'Year_{i+1}' for i in range(years)], columns=columns)

# Calculate PPA revenue for each year
for i in range(years):
    ppa_offers[f'PPA_Revenue_Year_{i+1}'] = ppa_offers.apply(
        lambda row: row['PPA_Price'] * ((1 + row['CPI']) ** i) * row['Capacity'] * annual_energy_production_mwh if i < row['Tenor'] else 0,
        axis=1
    )

# Calculate total revenue
for i in range(years):
    if i < 7:  # Years 1 to 7
        ppa_revenue = ppa_offers[f'PPA_Revenue_Year_{i+1}'].sum()
        spot_revenue = spot_prices[i] * (1 - ppa_offers['Capacity'].sum()) * annual_energy_production_mwh
    else:  # Years 8 to 10, no PPAs
        ppa_revenue = 0
        spot_revenue = spot_prices[i] * annual_energy_production_mwh
    total_revenue.loc[f'Year_{i+1}', 'PPA_Revenue'] = ppa_revenue
    total_revenue.loc[f'Year_{i+1}', 'Spot_Revenue'] = spot_revenue
    total_revenue.loc[f'Year_{i+1}', 'Total_Revenue'] = ppa_revenue + spot_revenue

# NPV calculation
total_revenue['Discount_Factor'] = [(1 / (1 + discount_rate) ** (i + 1)) for i in range(years)]
total_revenue['Discounted_Revenue'] = total_revenue['Total_Revenue'] * total_revenue['Discount_Factor']

# Calculate NPV
npv = total_revenue['Discounted_Revenue'].sum()

# Add NPV row
npv_row = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan, npv]], columns=total_revenue.columns, index=['NPV'])
total_revenue = pd.concat([total_revenue, npv_row])

# Prepare data for Dash DataTable
table_data = total_revenue.reset_index().rename(columns={'index': 'Year'})
table_data = table_data.round(2)  # Round values for better readability
ppa_offers_summary = ppa_offers[['PPA_Price', 'CPI', 'Capacity', 'Tenor']]

# Dash app setup
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Revenue Streams and NPV Dashboard"),
    dcc.Graph(
        id='revenue-graph',
        figure={
            'data': [
                go.Scatter(x=total_revenue.index[:-1], y=total_revenue['PPA_Revenue'][:-1], mode='lines+markers', name='PPA Revenue'),
                go.Scatter(x=total_revenue.index[:-1], y=total_revenue['Spot_Revenue'][:-1], mode='lines+markers', name='Spot Revenue'),
                go.Scatter(x=total_revenue.index[:-1], y=total_revenue['Total_Revenue'][:-1], mode='lines+markers', name='Total Revenue')
            ],
            'layout': go.Layout(
                title='Revenue Streams over 10 Years',
                xaxis={'title': 'Year'},
                yaxis={'title': 'Revenue ($)'},
                template='plotly_white'
            )
        }
    ),
    html.Div([
        html.H2("PPA Offers Summary"),
        DataTable(
            id='ppa-offers-table',
            columns=[{"name": i, "id": i} for i in ppa_offers_summary.columns],
            data=ppa_offers_summary.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_cell={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
        ),
    ]),
    html.Div([
        html.H2("Revenue Table"),
        DataTable(
            id='revenue-table',
            columns=[{"name": i, "id": i} for i in table_data.columns],
            data=table_data.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_cell={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
        ),
    ]),
    html.Div([
        html.H2("NPV Summary"),
        html.P(f"Net Present Value (NPV): ${npv:,.2f}", style={'fontSize': 20, 'fontWeight': 'bold'})
    ], style={'marginTop': 20})
])

if __name__ == '__main__':
    app.run_server(debug=True)